<a href="https://colab.research.google.com/github/Hexclan14/AutoMediQu/blob/main/Patient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import uuid
class Patient:
  def __init__(self, name , dateofbirth , place of residence , contact_number , medical_history = None , wantsservice = true):
    self.id = str(uuid.uuid4())
    self.name = name
    self.dateofbirth = dateofbirth
    self.placeofresidence = placeofresidence
    self.contact_number = contact_number
    self.medical_history = medical_history
    self.wantsservice = wantsservice
  def add_record(self, record):
    self.medical_history.append(record)
  def update_service(self , preference):
    self.wantsservice = preference
  def __str__(self):
    return f"Patient Name: {self.name}\nDate of Birth: {self.dateofbirth}\nPlace of Residence: {self.placeofresidence}\nContact Number: {self.contact_number}\n


import uuid
import datetime
from sqlalchemy import create_engine, Column, String, Date, Boolean, ForeignKey, Text, TIMESTAMP
from sqlalchemy.orm import declarative_base, sessionmaker

Base = declarative_base()

class PatientDB(Base):
    __tablename__ = "patients"
    patient_id = Column(String, primary_key=True)
    name = Column(String)
    date_of_birth = Column(Date)
    contact_number = Column(String)
    address = Column(Text)
    wants_services = Column(Boolean)

class MedicalRecordDB(Base):
    __tablename__ = "medical_records"
    record_id = Column(String, primary_key=True)
    patient_id = Column(String, ForeignKey("patients.patient_id"))
    record_date = Column(TIMESTAMP, default=datetime.datetime.utcnow)
    record_text = Column(Text)

# Database Connection and Session
engine = create_engine("sqlite:///patient_database.db")  # Change to your database URI
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

def get_patient_by_id(patient_id):
    """Retrieves a patient from the database by patient ID."""
    session = Session()
    try:
        return session.query(PatientDB).filter_by(patient_id=patient_id).first()
    finally:
        session.close()

def create_patient(patient_data):
    """Creates a new patient record in the database."""
    session = Session()
    try:
        new_patient = PatientDB(
            patient_id=str(uuid.uuid4()),
            name=patient_data["name"],
            date_of_birth=datetime.datetime.strptime(patient_data["date_of_birth"], "%Y-%m-%d").date(),
            contact_number=patient_data["contact_number"],
            address=patient_data["address"],
            wants_services=patient_data["wants_services"]
        )
        session.add(new_patient)
        session.commit()
        return new_patient.patient_id
    except Exception as e:
        session.rollback()
        print(f"Error creating patient: {e}")
        return None
    finally:
        session.close()

def update_patient(patient_id, patient_data):
    """Updates an existing patient record in the database."""
    session = Session()
    try:
        patient = session.query(PatientDB).filter_by(patient_id=patient_id).first()
        if patient:
            patient.name = patient_data.get("name", patient.name)
            patient.date_of_birth = datetime.datetime.strptime(patient_data.get("date_of_birth", patient.date_of_birth.strftime("%Y-%m-%d")), "%Y-%m-%d").date()
            patient.contact_number = patient_data.get("contact_number", patient.contact_number)
            patient.address = patient_data.get("address", patient.address)
            patient.wants_services = patient_data.get("wants_services", patient.wants_services)
            session.commit()
            return True
        else:
            return False
    except Exception as e:
        session.rollback()
        print(f"Error updating patient: {e}")
        return False
    finally:
        session.close()

def add_medical_record(patient_id, record_text):
    """Adds a medical record to a patient's history."""
    session = Session()
    try:
        new_record = MedicalRecordDB(
            record_id=str(uuid.uuid4()),
            patient_id=patient_id,
            record_text=record_text
        )
        session.add(new_record)
        session.commit()
        return True
    except Exception as e:
        session.rollback()
        print(f"Error adding medical record: {e}")
        return False
    finally:
        session.close()